# Code — imports & load cleaned data

In [ ]:


import pandas as pd, numpy as np
from pathlib import Path
PROC = Path("data/processed")
df = pd.read_csv(PROC / "trade_cleaned.csv")
df.shape

In [ ]:
df['grand_total_inr'] = df['total_value_inr'] + df['duty_paid_inr']
df['landed_cost_per_unit_inr'] = df.apply(lambda r: r['grand_total_inr']/r['quantity'] if r['quantity']>0 else None, axis=1)


In [ ]:
def assign_category(desc):
    if not isinstance(desc, str): return 'Others'
    d = desc.upper()
    if 'GLASS' in d: return 'Glass'
    if 'WOOD' in d or 'WOODEN' in d: return 'Wooden'
    if 'STEEL' in d or 'SS' in d: return 'Steel'
    if 'PLASTIC' in d or 'PVC' in d: return 'Plastic'
    return 'Others'

df['category'] = df['goods_description'].apply(assign_category)
df['category'].value_counts().head(10)


In [ ]:
df['duty_pct'] = df.apply(lambda r: r['duty_paid_inr']/r['total_value_inr'] if r['total_value_inr']>0 else None, axis=1)
mu = df['duty_pct'].mean(skipna=True)
sigma = df['duty_pct'].std(skipna=True)
df['duty_anomaly'] = df['duty_pct'].apply(lambda x: True if pd.notna(x) and (abs(x-mu) > 2*sigma) else False)
df[['duty_pct','duty_anomaly']].head()


In [ ]:
FINAL = PROC / "trade_features.csv"
cols_to_keep = ['date_of_shipment','year','month','hsn_code','goods_description',
                'quantity','unit_standard','unit_price_usd_extracted','total_value_inr','duty_paid_inr',
                'grand_total_inr','landed_cost_per_unit_inr','category','model_number_extracted','capacity_extracted']
df[cols_to_keep].to_csv(FINAL, index=False)
print("Saved", FINAL)


next steps: load the trade_features.csv into SQL (use your load_to_mysql.py), run macro_trends.sql etc.